[View in Colaboratory](https://colab.research.google.com/github/nikhilsingh291/eip_theinkers/blob/master/NIKHIL_SINGH_Session7.ipynb)

datalab


#Deep Learning for Visual Question Answering


![alt text](https://avisingh599.github.io/images/vqa/sample_results.jpg)

This is about answering general english questions based on images by deep learning.

I will be using LSTM+CNN approach to solve this problem and get a better accuracy.

In [2]:
!mkdir data
%cd data


/content/data


In [3]:
# Downloads the training and validation sets from visualqa.org. 

!wget http://visualqa.org/data/mscoco/vqa/Questions_Train_mscoco.zip
!wget http://visualqa.org/data/mscoco/vqa/Questions_Val_mscoco.zip
!wget http://visualqa.org/data/mscoco/vqa/Annotations_Train_mscoco.zip
!wget http://visualqa.org/data/mscoco/vqa/Annotations_Val_mscoco.zip

!unzip \*.zip

--2018-06-06 17:09:21--  http://visualqa.org/data/mscoco/vqa/Questions_Train_mscoco.zip
Resolving visualqa.org (visualqa.org)... 128.173.88.40
Connecting to visualqa.org (visualqa.org)|128.173.88.40|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21985607 (21M) [application/zip]
Saving to: ‘Questions_Train_mscoco.zip’

Questions_Train_msc 100%[===================>]  20.97M  9.63MB/s    in 2.2s    

2018-06-06 17:09:24 (9.63 MB/s) - ‘Questions_Train_mscoco.zip’ saved [21985607/21985607]

--2018-06-06 17:09:25--  http://visualqa.org/data/mscoco/vqa/Questions_Val_mscoco.zip
Resolving visualqa.org (visualqa.org)... 128.173.88.40
Connecting to visualqa.org (visualqa.org)|128.173.88.40|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10594497 (10M) [application/zip]
Saving to: ‘Questions_Val_mscoco.zip’

Questions_Val_mscoc 100%[===================>]  10.10M  5.51MB/s    in 1.8s    

2018-06-06 17:09:27 (5.51 MB/s) - ‘Questions_Val_mscoco.zip’

In [4]:
!ls
%cd ..
!ls
!mkdir features

Annotations_Train_mscoco.zip
Annotations_Val_mscoco.zip
mscoco_train2014_annotations.json
mscoco_val2014_annotations.json
MultipleChoice_mscoco_train2014_questions.json
MultipleChoice_mscoco_val2014_questions.json
OpenEnded_mscoco_train2014_questions.json
OpenEnded_mscoco_val2014_questions.json
Questions_Train_mscoco.zip
Questions_Val_mscoco.zip
/content
data  datalab


In [5]:
%cd features
!ls
# Downloads and unzips the VGG features computed on the COCO dataset. 

!wget http://cs.stanford.edu/people/karpathy/deepimagesent/coco.zip
!unzip coco.zip -d .

/content/features
--2018-06-06 17:14:32--  http://cs.stanford.edu/people/karpathy/deepimagesent/coco.zip
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cs.stanford.edu/people/karpathy/deepimagesent/coco.zip [following]
--2018-06-06 17:14:32--  https://cs.stanford.edu/people/karpathy/deepimagesent/coco.zip
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 764830639 (729M) [application/zip]
Saving to: ‘coco.zip’

coco.zip             47%[========>           ] 344.96M  11.9MB/s    eta 30s    

coco.zip            100%[===================>] 729.40M  14.2MB/s    in 59s     

2018-06-06 17:15:31 (12.3 MB/s) - ‘coco.zip’ saved [764830639/764830639]

Archive:  coco.zip
  inflating: ./coco/dataset.json     
  inflating: ./coco/readme.txt       
  inflating: ./coco/vgg_feats.mat    


In [7]:
!ls
%cd ..

coco  coco.zip
/content


In [9]:
!pwd
!ls
!pip install spacy

/content
data  datalab  features
  Using cached https://files.pythonhosted.org/packages/3c/31/e60f88751e48851b002f78a35221d12300783d5a43d4ef12fbf10cca96c3/spacy-2.0.11.tar.gz
  Using cached https://files.pythonhosted.org/packages/5e/31/c8c1ecafa44db30579c8c457ac7a0f819e8b1dbc3e58308394fff5ff9ba7/murmurhash-0.28.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/f8/9e/273fbea507de99166c11cd0cb3fde1ac01b5bc724d9a407a2f927ede91a1/cymem-1.31.2.tar.gz
  Using cached https://files.pythonhosted.org/packages/1b/ac/7c17b1fd54b60972785b646d37da2826311cca70842c011c4ff84fbe95e0/preshed-1.0.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/55/fd/e9f36081e6f53699943381858848f3b4d759e0dd03c43b98807dde34c252/thinc-6.10.2.tar.gz
  Using cached https://files.pythonhosted.org/packages/9e/9b/62c60d2f5bc135d2aa1d8c8a86aaf84edb719a59c7f11a4316259e61a298/plac-0.9.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ac/aa/9b065a76b9af472437a0059f77e8f962fe35

 - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done
  Stored in directory: /content/.cache/pip/wheels/fb/00/28/75c85d5135e7d9a100639137d1847d41e914ed16c962d467e4
  Running setup.py bdist_wheel for murmurhash ... - \ | / done
  Stored in directory: /content/.cache/pip/wheels/b8/94/a4/f69f8664cdc1098603df44771b7fec5fd1b3d8364cdd83f512
  Running setup.py bdist_wheel for cymem ... - \ | done
  Stored in directory: /content/.cache/pip/wheels/55/8d/4a/f6328252aa2aaec0b1cb906fd96a1566d77f0f67701071ad13
  Running setup.py bdist_wheel for preshed ... - \ | / - \ | done
  Stored in directory: /content/.cache/pip/wheels/8f/85/06/2d132fb649a6bbcab22487e4147880a55b0dd0f4b18fdfd6b5
  Running setup.py bdist_wheel for thinc ... - \ | / - \ | / - \ |

 - \ | / done
  Stored in directory: /content/.cache/pip/wheels/f8/b1/86/c92e4d36b690208fff8471711b85eaa6bc6d19860a86199a09
  Running setup.py bdist_wheel for msgpack-python ... - \ | / done
  Stored in directory: /content/.cache/pip/wheels/d5/de/86/7fa56fda12511be47ea0808f3502bc879df4e63ab168ec0406
Successfully built spacy murmurhash cymem preshed thinc pathlib ujson dill regex wrapt cytoolz msgpack-python


In [10]:
!python3 -m spacy download en


    100% |████████████████████████████████| 37.4MB 60.1MB/s 
  Running setup.py install for en-core-web-sm ... - \ | done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')

Cloning into 'visual_q_ans'...
remote: Counting objects: 19, done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 19 (delta 7), reused 12 (delta 2), pack-reused 0
Unpacking objects: 100% (19/19), done.
data  datalab  features  visual_q_ans


In [20]:
'''!ls
%cd
!ls
!rm -rf visual_q_ans/
!ls
'''
!pip3 install progressbar

  Using cached https://files.pythonhosted.org/packages/0f/e1/f30b72ecabba259c7c38dd0eb944a173effced3fd7de2c9c2902bd6f649a/progressbar-2.3.tar.gz
    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/tmp/pip-install-8zw8qd5w/progressbar/setup.py", line 5, in <module>
        import progressbar
      File "/tmp/pip-install-8zw8qd5w/progressbar/progressbar/__init__.py", line 59, in <module>
        from progressbar.widgets import *
      File "/tmp/pip-install-8zw8qd5w/progressbar/progressbar/widgets.py", line 121, in <module>
        class FileTransferSpeed(Widget):
      File "/usr/lib/python3.6/abc.py", line 133, in __new__
        cls = super().__new__(mcls, name, bases, namespace, **kwargs)
    ValueError: 'format' in __slots__ conflicts with class variable
    
    ----------------------------------------
Command "python setup.py egg_info" failed with error code 1 in /tmp/pip-install-

In [18]:
"""import operator
from itertools import zip_longest
from collections import defaultdict

def selectFrequentAnswers(questions_train, answers_train, images_train, maxAnswers):
	answer_fq= defaultdict(int)
	#build a dictionary of answers
	for answer in answers_train:
		answer_fq[answer] += 1

	sorted_fq = sorted(answer_fq.items(), key=operator.itemgetter(1), reverse=True)[0:maxAnswers]
	top_answers, top_fq = zip(*sorted_fq)
	new_answers_train=[]
	new_questions_train=[]
	new_images_train=[]
	#only those answer which appear int he top 1K are used for training
	for answer,question,image in zip(answers_train, questions_train, images_train):
		if answer in top_answers:
			new_answers_train.append(answer)
			new_questions_train.append(question)
			new_images_train.append(image)

	return (new_questions_train,new_answers_train,new_images_train)

def grouper(iterable, n, fillvalue=None):
    args = [iter(iterable)] * n
    return izip_longest(*args, fillvalue=fillvalue)
    """

!git clone http://github.com/nikhilsingh291/visual_q_ans.git
!ls
%cd visual_q_ans/
!ls

!python dumpText.py

Cloning into 'visual_q_ans'...
remote: Counting objects: 22, done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 22 (delta 9), reused 15 (delta 4), pack-reused 0
Unpacking objects: 100% (22/22), done.
data  datalab  features  visual_q_ans
/content/visual_q_ans
dumpText.py	     features.py  own_image.py	utils.py
extract_features.py  mainpy.py	  README.md
Traceback (most recent call last):
  File "dumpText.py", line 3, in <module>
    import progressbar
ModuleNotFoundError: No module named 'progressbar'


In [0]:
import numpy as np
from keras.utils import np_utils


def get_questions_tensor_timeseries(questions, nlp, timesteps):
	'''
	Returns a time series of word vectors for tokens in the question

	Input:
	questions: list of unicode objects
	nlp: an instance of the class English() from spacy.en
	timesteps: the number of 

	Output:
	A numpy ndarray of shape: (nb_samples, timesteps, word_vec_dim)
	'''
	assert not isinstance(questions, basestring)
	nb_samples = len(questions)
	word_vec_dim = nlp(questions[0])[0].vector.shape[0]
	questions_tensor = np.zeros((nb_samples, timesteps, word_vec_dim))
	for i in xrange(len(questions)):
		tokens = nlp(questions[i])
		for j in xrange(len(tokens)):
			if j<timesteps:
				questions_tensor[i,j,:] = tokens[j].vector

	return questions_tensor

def get_questions_matrix_sum(questions, nlp):
	'''
	Sums the word vectors of all the tokens in a question
	
	Input:
	questions: list of unicode objects
	nlp: an instance of the class English() from spacy.en

	Output:
	A numpy array of shape: (nb_samples, word_vec_dim)	
	'''
	assert not isinstance(questions, basestring)
	nb_samples = len(questions)
	word_vec_dim = nlp(questions[0])[0].vector.shape[0]
	questions_matrix = np.zeros((nb_samples, word_vec_dim))
	for i in xrange(len(questions)):
		tokens = nlp(questions[i])
		for j in xrange(len(tokens)):
			questions_matrix[i,:] += tokens[j].vector

	return questions_matrix

def get_answers_matrix(answers, encoder):
	'''
	Converts string objects to class labels

	Input:
	answers:	a list of unicode objects
	encoder:	a scikit-learn LabelEncoder object

	Output:
	A numpy array of shape (nb_samples, nb_classes)
	'''
	assert not isinstance(answers, basestring)
	y = encoder.transform(answers) #string to numerical class
	nb_classes = encoder.classes_.shape[0]
	Y = np_utils.to_categorical(y, nb_classes)
	return Y

def get_images_matrix(img_coco_ids, img_map, VGGfeatures):
	'''
	Gets the 4096-dimensional CNN features for the given COCO
	images
	
	Input:
	img_coco_ids: 	A list of strings, each string corresponding to
				  	the MS COCO Id of the relevant image
	img_map: 		A dictionary that maps the COCO Ids to their indexes 
					in the pre-computed VGG features matrix
	VGGfeatures: 	A numpy array of shape (nb_dimensions,nb_images)

	Ouput:
	A numpy matrix of size (nb_samples, nb_dimensions)
	'''
	assert not isinstance(img_coco_ids, basestring)
	nb_samples = len(img_coco_ids)
	nb_dimensions = VGGfeatures.shape[0]
	image_matrix = np.zeros((nb_samples, nb_dimensions))
	for j in xrange(len(img_coco_ids)):
		image_matrix[j,:] = VGGfeatures[:,img_map[img_coco_ids[j]]]

	return image_matrix

In [0]:
import numpy as np
import scipy.io
import sys

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, Reshape
from keras.layers import Merge
from keras.layers.recurrent import LSTM
from keras.utils import np_utils, generic_utils
from keras.callbacks import ModelCheckpoint, RemoteMonitor

from sklearn.externals import joblib
from sklearn import preprocessing

#from spacy.en import English

#from utils import grouper, selectFrequentAnswers
#from features import get_images_matrix, get_answers_matrix, get_questions_tensor_timeseries

In [7]:
!ls


datalab


In [27]:
word_vec_dim= 300
img_dim = 4096
max_len = 30
nb_classes = 1000

questions_train = open('../data/preprocessed/questions_train2014.txt', 'r').read().decode('utf8').splitlines()
questions_lengths_train = open('../data/preprocessed/questions_lengths_train2014.txt', 'r').read().decode('utf8').splitlines()
answers_train = open('../data/preprocessed/answers_train2014_modal.txt', 'r').read().decode('utf8').splitlines()
images_train = open('../data/preprocessed/images_train2014.txt', 'r').read().decode('utf8').splitlines()
vgg_model_path = '../features/coco/vgg_feats.mat'

FileNotFoundError: ignored